## Need to change to zetasql but no option in beam_sql magic

In [ ]:
import json
import os
import typing
import datetime

PARENT_DIR = os.path.dirname(os.getcwd())

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive import interactive_runner

In [ ]:
class EventLog(typing.NamedTuple):
    ip: str
    id: str
    lat: float
    lng: float
    user_agent: str
    age_bracket: str
    opted_into_marketing: bool
    http_request: str
    http_response: int
    file_size_bytes: int
    event_datetime: str
    event_ts: int

beam.coders.registry.register_coder(EventLog, beam.coders.RowCoder)

def parse_json(element: str):
    row = json.loads(element)
    # lat/lng sometimes empty string
    if not row["lat"] or not row["lng"]:
        row = {**row, **{"lat": -1, "lng": -1}}
    return EventLog(**row)


def format_timestamp(element: EventLog):
    event_ts = datetime.datetime.fromisoformat(element.event_datetime)
    temp_dict = element._asdict()
    temp_dict["event_datetime"] = datetime.datetime.strftime(
        event_ts, "%Y-%m-%d %H:%M:%S"
    )
    return EventLog(**temp_dict)


In [ ]:
p = beam.Pipeline(interactive_runner.InteractiveRunner())

events = (
    p
    | "Read from files"
    >> beam.io.ReadFromText(
        file_pattern=os.path.join(os.path.join(PARENT_DIR, "inputs", "*.out"))
    )
    | "Parse elements" >> beam.Map(parse_json).with_output_types(EventLog)
    | "Format timestamp" >> beam.Map(format_timestamp).with_output_types(EventLog)
)

ib.show(events, n=5)

In [ ]:
%load_ext apache_beam.runners.interactive.sql.beam_sql_magics

In [ ]:
%%beam_sql -o output
SELECT
    STRING(window_start) AS start_time,
    STRING(window_end) AS end_time,
    COUNT(*) AS page_views
FROM
    TUMBLE(
        (SELECT TIMESTAMP(event_datetime) AS ts FROM events),
        DESCRIPTOR(ts),
        'INTERVAL 1 MINUTE')
GROUP BY
    window_start, window_end